# using py spark in vm

importing find spark and pyspark

In [1]:
import findspark
findspark.init('/usr/local/spark')
import pyspark

Now start SparkSession

In [2]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("Python Spark SQL example").getOrCreate()

Create dataFrame from data source -csv file

In [3]:
customerDF=spark.read.load("/home/hduser/Downloads/sharedfolder/Customer.txt",format="csv",sep="\t",interSchema="true",header='true')

data transdormation (displaying data in different ways)

In [4]:
customerDF.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)
 |-- customer_zipcode: string (nullable = true)



In [5]:
customerDF.select("customer_name").show()

+----------------+
|   customer_name|
+----------------+
|     Mary Torres|
|      Jose Haley|
|      Mary Smith|
|  Richard Maddox|
|  Margaret Booth|
|  Mary Henderson|
|     Lisa Walker|
|   Jonathan Hill|
|Carolyn Sheppard|
|    Mary Mendoza|
|   Michael Smith|
|    James Holmes|
|     Mary Dawson|
|    Adam Marquez|
|    Gloria Smith|
|       Mary Webb|
|  Nancy Alvarado|
|  Russell Flores|
|    Denise Smith|
|  Jose Dickerson|
+----------------+
only showing top 20 rows



In [6]:
customerDF.select(customerDF["customer_name"],customerDF['customer_city']).show()

+----------------+-------------+
|   customer_name|customer_city|
+----------------+-------------+
|     Mary Torres|       Caguas|
|      Jose Haley|     Columbus|
|      Mary Smith|      Houston|
|  Richard Maddox|       Caguas|
|  Margaret Booth|    Arlington|
|  Mary Henderson|       Caguas|
|     Lisa Walker|       Caguas|
|   Jonathan Hill|      Phoenix|
|Carolyn Sheppard|Pompano Beach|
|    Mary Mendoza|       Caguas|
|   Michael Smith|       Caguas|
|    James Holmes|     Hilliard|
|     Mary Dawson|       Caguas|
|    Adam Marquez|  San Antonio|
|    Gloria Smith|       Caguas|
|       Mary Webb|   San Marcos|
|  Nancy Alvarado|     Flushing|
|  Russell Flores|       Caguas|
|    Denise Smith|    Rego Park|
|  Jose Dickerson|         Mesa|
+----------------+-------------+
only showing top 20 rows



In [7]:
customerDF.filter(customerDF["customer_state"] == 'CA').show()

+-----------+----------------+---------------+--------------+----------------+
|customer_id|   customer_name|  customer_city|customer_state|customer_zipcode|
+-----------+----------------+---------------+--------------+----------------+
|       5577|      Mary Smith|        Modesto|            CA|           95350|
|       1745|      Mary Smith|Rowland Heights|            CA|           91748|
|      11444|Kathleen Patrick|      San Diego|            CA|           92109|
|       8846|    Thomas Smith|          Indio|            CA|           92201|
|       6237|  Bobby Anderson|       El Cajon|            CA|           92020|
|       4085|       Mary Carr|  Panorama City|            CA|           91402|
|       8705|  Patricia Smith|       Stockton|            CA|           95207|
|       3669|       Mary Soto| San Bernardino|            CA|           92410|
|       6101|      Mary Smith|    Los Angeles|            CA|           90033|
|      11697|  Jessica Thomas|  Laguna Niguel|      

In [8]:
customerDF.groupby("customer_state").count().show()

+--------------+-----+
|customer_state|count|
+--------------+-----+
|            AZ|   19|
|            SC|    2|
|            LA|    7|
|            MN|    1|
|            NJ|   19|
|            DC|    4|
|            OR|    4|
|            VA|   14|
|            RI|    2|
|            KY|    1|
|            MI|   28|
|            NV|   16|
|            WI|    9|
|            ID|    2|
|            CA|  187|
|            CT|    8|
|            NC|   19|
|            MD|   19|
|            DE|    1|
|            MO|   13|
+--------------+-----+
only showing top 20 rows



create temp view for running SQL queries on the dataframe

In [9]:
customerDF.createOrReplaceTempView("customers")

SQL can be run on DataFrame that are registered as temp view

In [10]:
spark.sql("SELECT customer_state, count(*) as state_count FROM customers GROUP BY customer_state HAVING state_count>=50").show()

+--------------+-----------+
|customer_state|state_count|
+--------------+-----------+
|            CA|        187|
|            NY|         79|
|            TX|         62|
|            PR|        505|
+--------------+-----------+



we should store the query output in different variable

In [11]:
cstateCount50 = spark.sql("SELECT customer_state, count(*) as state_count FROM customers GROUP BY customer_state HAVING state_count>=50")

In [12]:
type(cstateCount50)

pyspark.sql.dataframe.DataFrame

In [13]:
cstateCount50.show()

+--------------+-----------+
|customer_state|state_count|
+--------------+-----------+
|            CA|        187|
|            NY|         79|
|            TX|         62|
|            PR|        505|
+--------------+-----------+



In [14]:
cstateCount50.printSchema()

root
 |-- customer_state: string (nullable = true)
 |-- state_count: long (nullable = false)



In [15]:
cstateCount50.coalesce(1).write.parquet("cStateOutput1.parquet")

Since the data is small we can coalesce all the partition into one column ans write it into cStateOutput1.parquet file, for large datasets it is expensive

Alternatively the following syntax is also allowed.
we can use the parquet synatx since it is the default dormat used by sparksql

cstateCount50.coalesce(1).write.save("cStateOutput1.parquet",format='parquet')

cstateCount50.coalesce(1).write.save("cStateOutput1.parquet")

cstateCount50.coalesce(1).write.save("cStateOutput1.json",format="json")

cstateCount50.coalesce(1).write.json("cStateOutput1.json")

In [16]:
cstateCount50.coalesce(1).write.json("cStateOutput1.json")

In [17]:
readingPQ=spark.read.parquet('cStateOutput1.parquet')

In [18]:
readingPQ.show()

+--------------+-----------+
|customer_state|state_count|
+--------------+-----------+
|            CA|        187|
|            NY|         79|
|            TX|         62|
|            PR|        505|
+--------------+-----------+



Now create dataFrame from datasource -json file

In [19]:
productDF = spark.read.load("/home/hduser/Downloads/sharedfolder/products.json", format = 'json')

In [20]:
productDF.printSchema()

root
 |-- category_id: long (nullable = true)
 |-- customer_id: long (nullable = true)
 |-- product_category: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- product_price: double (nullable = true)
 |-- product_quantity: long (nullable = true)
 |-- salestxn_id: long (nullable = true)



In [21]:
productDF.select(productDF["product_name"], productDF['product_price']).show()

+--------------------+-------------+
|        product_name|product_price|
+--------------------+-------------+
|O'Brien Men's Neo...|        49.98|
|O'Brien Men's Neo...|        49.98|
|Under Armour Wome...|        31.99|
|O'Brien Men's Neo...|        49.98|
|Pelican Sunstream...|       199.99|
|Nike Men's CJ Eli...|       129.99|
|Diamondback Women...|       299.98|
|Field & Stream Sp...|       399.98|
|Perfect Fitness P...|        59.99|
|Nike Men's CJ Eli...|       129.99|
|Pelican Sunstream...|       199.99|
|Nike Men's CJ Eli...|       129.99|
|Diamondback Women...|       299.98|
|Nike Men's CJ Eli...|       129.99|
|Nike Men's Dri-FI...|         50.0|
|O'Brien Men's Neo...|        49.98|
|O'Brien Men's Neo...|        49.98|
|Nike Men's Dri-FI...|         50.0|
|Diamondback Women...|       299.98|
|Under Armour Girl...|        39.99|
+--------------------+-------------+
only showing top 20 rows



In [22]:
productDF.groupby('product_category').count().show()

+-------------------+-----+
|   product_category|count|
+-------------------+-----+
|  Training by Sport|    5|
|   Men's Golf Clubs|   21|
|   Camping & Hiking|   44|
|Fitness Accessories|   47|
|         Golf Shoes|    6|
|         Basketball|   36|
|        Electronics|   48|
|          Team Shop|  162|
|      Men's Apparel| 2085|
|  Bike & Skate Shop| 1377|
|  Golf Bags & Carts|   89|
|    As Seen on  TV!| 2399|
|       Boxing & MMA|  115|
| Hunting & Shooting| 1785|
|Baseball & Softball|    4|
|       Golf Apparel|   51|
| Women's Golf Clubs|   57|
|      Shop By Sport|   26|
|            Fishing| 1953|
|        Accessories|  110|
+-------------------+-----+
only showing top 20 rows

